In [65]:
import pandas as pd
import pandasql as ps
import numpy as np

In [12]:
teams = pd.read_csv('./Team_nodes.csv')

In [58]:
teams.columns

Index(['postGame_side', 'postGame_goals_scored', 'postGame_goals_conceded',
       'postGame_index', 'postGame_total_shots', 'postGame_total_xg',
       'postGame_total_big_chances', 'postGame_total_key_passes',
       'competitionId', 'gameId', 'teamId', 'preGame_steph',
       'preGame_quality_overall', 'preGame_quality_attack',
       'preGame_quality_midfield', 'preGame_quality_defence',
       'preGame_proportion_pts', 'gameDate', 'teamName', 'competitionName',
       'nodeId', 'nodeName'],
      dtype='object')

In [103]:
query = """
    select * 
    from teams
    order by gameDate, teamName
"""
teams = ps.sqldf(query)

In [129]:
query = "select teamName,"
_columns = ['postGame_side',
       'competitionId', 'gameId', 'teamId', 'preGame_steph',
       'preGame_quality_overall', 'preGame_quality_attack',
       'preGame_quality_midfield', 'preGame_quality_defence',
       'preGame_proportion_pts', 'gameDate', 'teamName', 'competitionName',
       'nodeId', 'nodeName']
post_columns = ['postGame_goals_scored', 'postGame_goals_conceded',
       'postGame_index', 'postGame_total_shots', 'postGame_total_xg',
       'postGame_total_big_chances', 'postGame_total_key_passes']
for column in _columns:
    q_plus = f"""
        {column},
    """
    query += q_plus
for i, column in enumerate(post_columns):
    try:
        columns[i+1]
        sep = ',' 
    except IndexError:
        sep = ''
    q_plus = f"""
                {column},
                avg({column}) over (
                    partition by teamName rows between unbounded preceding
                    and 1 preceding
                ) as agg_{column}{sep}
            """
    query += q_plus

query += """
    from teams
    order by gameDate, teamName
"""
features = ps.sqldf(query)
features.to_csv('team_nodes_plus_features.csv')